In [1]:
import utils as u
import torch
import torch.distributed as dist
import numpy as np
import time
import random
import sbm_dl as sbm
import link_pred_tasker as lpt
import models as mls
import egcn_o
import splitter as sp
import Cross_Entropy as ce
import trainer as tr

args = u.Namespace({
    "num_hist_steps": 0, 
    "adj_mat_time_window": 1,
    "smart_neg_sampling": True, 
    "negative_mult_training": 50,
    "negative_mult_test": 100,
    'use_2_hot_node_feats': False,
    'use_1_hot_node_feats': True,
    'aggr_time': 1,
    'train_proportion': 1,
    'dev_proportion': 0,
    'task':'link_pred'
})  

args.data_loading_params = {'batch_size': 1,'num_workers': 8}
args.sbm_args = {'folder': './data/',  'edges_file': 'GEO.csv',  'aggr_time': 1 ,'feats_per_node': 3}
args.gcn_parameters = {'feats_per_node': 100,  'feats_per_node_min': 50,  'feats_per_node_max': 256,  'layer_1_feats': 100,
                       'layer_1_feats_min': 10,  'layer_1_feats_max': 200,  'layer_2_feats': 100,  'layer_2_feats_same_as_l1': True,
                       'k_top_grcu': 200,  'num_layers': 2,  'lstm_l1_layers': 1,  'lstm_l1_feats': 100, 'lstm_l1_feats_min': 10,
                       'lstm_l1_feats_max': 200,  'lstm_l2_layers': 1,  'lstm_l2_feats': None, 'lstm_l2_feats_same_as_l1': True,
                       'cls_feats': 100, 'cls_feats_min': 64,  'cls_feats_max': 800}

dataset = sbm.sbm_dataset(args)
tasker = lpt.Link_Pred_Tasker(args,dataset)
splitter = sp.splitter(args,tasker)
gcn_args = u.Namespace(args.gcn_parameters)
gcn_args.feats_per_node = tasker.feats_per_node
gcn = egcn_o.EGCN(gcn_args, activation = torch.nn.RReLU())

[14/May/2024 21:24:36] INFO - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[14/May/2024 21:24:36] INFO - NumExpr defaulting to 8 threads.
TIME tensor(2) tensor(0)


/home/yrc/STAD/taskers_utils.py:98: UserWarning: torch.sparse.SparseTensor(indices, values, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:593.)
  out = torch.sparse.FloatTensor(idx.t(),vals).coalesce()
/home/yrc/STAD/utils.py:69: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:605.)
  return torch.sparse.LongTensor(adj['idx'].t(),


Dataset splits sizes:  train 2


In [2]:
num_nodes = dataset.num_nodes
def ignore_batch_dim(adj):
    adj['idx'] = adj['idx'][0]
    adj['vals'] = adj['vals'][0]
    return adj
def prepare_sample(sample):
    sample = u.Namespace(sample)
    for i,adj in enumerate(sample.hist_adj_list):
        adj = u.sparse_prepare_tensor(adj,torch_size = [num_nodes])
        sample.hist_adj_list[i] = adj
        
        nodes = splitter.tasker.prepare_node_feats(sample.hist_ndFeats_list[i])
        
        sample.hist_ndFeats_list[i] = nodes
        node_mask = sample.node_mask_list[i]
        sample.node_mask_list[i] = node_mask.t()
        
    label_sp = ignore_batch_dim(sample.label_sp)
    
    label_sp['idx'] = label_sp['idx'].t()
    
    label_sp['vals'] = label_sp['vals'].type(torch.long)
    sample.label_sp = label_sp
    
    return sample

spatial = []

for s in splitter.train:
    s = prepare_sample(s)
    nodes_embs = gcn(s.hist_adj_list,
                     s.hist_ndFeats_list,
                     s.node_mask_list)
    print(nodes_embs)
    spatial.append(nodes_embs)
spatial

Ahat shape: torch.Size([5592, 5592])
node_embs shape: torch.Size([5592, 5351])
GCN_weights shape: torch.Size([5351, 100])
Ahat shape: torch.Size([5592, 5592])
node_embs shape: torch.Size([5592, 100])
GCN_weights shape: torch.Size([100, 100])
tensor([[ 0.0056,  0.0051, -0.0016,  ...,  0.0143,  0.0051, -0.0013],
        [ 0.0053,  0.0049, -0.0012,  ...,  0.0143,  0.0033, -0.0010],
        [ 0.0048,  0.0042, -0.0019,  ...,  0.0137,  0.0036, -0.0007],
        ...,
        [ 0.0051,  0.0046, -0.0024,  ...,  0.0137,  0.0043, -0.0006],
        [ 0.0051,  0.0053, -0.0019,  ...,  0.0133,  0.0048, -0.0012],
        [ 0.0056,  0.0049, -0.0021,  ...,  0.0135,  0.0047, -0.0006]],
       grad_fn=<RreluWithNoiseBackward0>)
Ahat shape: torch.Size([5592, 5592])
node_embs shape: torch.Size([5592, 5351])
GCN_weights shape: torch.Size([5351, 100])
Ahat shape: torch.Size([5592, 5592])
node_embs shape: torch.Size([5592, 100])
GCN_weights shape: torch.Size([100, 100])
tensor([[ 0.0060,  0.0049, -0.0014,  ...

[tensor([[ 0.0056,  0.0051, -0.0016,  ...,  0.0143,  0.0051, -0.0013],
         [ 0.0053,  0.0049, -0.0012,  ...,  0.0143,  0.0033, -0.0010],
         [ 0.0048,  0.0042, -0.0019,  ...,  0.0137,  0.0036, -0.0007],
         ...,
         [ 0.0051,  0.0046, -0.0024,  ...,  0.0137,  0.0043, -0.0006],
         [ 0.0051,  0.0053, -0.0019,  ...,  0.0133,  0.0048, -0.0012],
         [ 0.0056,  0.0049, -0.0021,  ...,  0.0135,  0.0047, -0.0006]],
        grad_fn=<RreluWithNoiseBackward0>),
 tensor([[ 0.0060,  0.0049, -0.0014,  ...,  0.0134,  0.0053, -0.0008],
         [ 0.0055,  0.0046, -0.0032,  ...,  0.0135,  0.0046, -0.0012],
         [ 0.0054,  0.0043, -0.0030,  ...,  0.0135,  0.0054, -0.0014],
         ...,
         [ 0.0046,  0.0043, -0.0031,  ...,  0.0135,  0.0055, -0.0007],
         [ 0.0053,  0.0044, -0.0020,  ...,  0.0137,  0.0048, -0.0007],
         [ 0.0055,  0.0040, -0.0029,  ...,  0.0141,  0.0044, -0.0011]],
        grad_fn=<RreluWithNoiseBackward0>)]

In [3]:
import pandas as pd
for i in range(len(spatial)):
    reshaped_tensor = torch.squeeze(spatial[i]).view(5592, 100)
    numpy_array = reshaped_tensor.detach().numpy()
    final = pd.DataFrame(numpy_array)
    final.to_csv("stage_"+str(i)+"_GEO.csv",index=None,columns=None)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import math

class LayerNorm(nn.Module):
    def __init__(self, size, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.eps = eps
        self.a_2 = nn.Parameter(torch.ones(size))
        self.b_2 = nn.Parameter(torch.zeros(size))

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class MLP(nn.Module):
    def __init__(self, in_size, mid_size, out_size, dropout_r=0., use_relu=True):
        super(MLP, self).__init__()

        self.fc = nn.Sequential(
            nn.Linear(in_size, mid_size,dtype=torch.float),
            nn.ReLU(inplace=True) if use_relu else nn.Identity(),
            nn.Dropout(dropout_r) if dropout_r > 0 else nn.Identity(),
            nn.Linear(mid_size, out_size,dtype=torch.float)
        )

    def forward(self, x):
        return self.fc(x)

class MHAtt(nn.Module):
    def __init__(self, config):
        super(MHAtt, self).__init__()
        self.config = config

        self.linear_v = nn.Linear(config.Feature1_SIZE, config.HIDDEN_SIZE,dtype=torch.float)
        self.linear_k = nn.Linear(config.Feature2_SIZE, config.HIDDEN_SIZE,dtype=torch.float)
        self.linear_q = nn.Linear(config.Feature1_SIZE, config.HIDDEN_SIZE,dtype=torch.float)
        self.linear_merge = nn.Linear(config.HIDDEN_SIZE, config.Feature1_SIZE,dtype=torch.float)

        self.dropout = nn.Dropout(config.DROPOUT_R)

    def forward(self, v, k, q, mask):
        n_batches = q.size(0)

        v = self.linear_v(v).view(
            n_batches,
            -1,
            self.config.MULTI_HEAD,
            self.config.HIDDEN_SIZE_HEAD
        ).transpose(1, 2)

        k = self.linear_k(k).view(
            n_batches,
            -1,
            self.config.MULTI_HEAD,
            self.config.HIDDEN_SIZE_HEAD
        ).transpose(1, 2)

        q = self.linear_q(q).view(
            n_batches,
            -1,
            self.config.MULTI_HEAD,
            self.config.HIDDEN_SIZE_HEAD
        ).transpose(1, 2)

        atted = self.att(v, k, q, mask)
        atted = atted.transpose(1, 2).contiguous().view(
            n_batches,
            -1,
            self.config.HIDDEN_SIZE
        )
        atted = self.linear_merge(atted)

        return atted

    def att(self, value, key, query, mask):
        d_k = query.size(-1)

        scores = torch.matmul(
            query, key.transpose(-2, -1)
        ) / math.sqrt(d_k)

        if mask is not None:
            scores = scores.masked_fill(mask, -1e9)

        att_map = F.softmax(scores, dim=-1)
        att_map = self.dropout(att_map)

        return torch.matmul(att_map, value)

class SGA(nn.Module):
    def __init__(self, config):
        super(SGA, self).__init__()

        self.multi_head_attention1 = MHAtt(config).float()
        # self.multi_head_attention2 = MHAtt(config).float()
        self.feed_forward = MLP(config.Feature1_SIZE, config.FF_SIZE, config.Feature1_SIZE)

        self.dropout1 = nn.Dropout(config.DROPOUT_R)
        self.layer_norm1 = LayerNorm(config.Feature1_SIZE)

        # self.dropout2 = nn.Dropout(config.DROPOUT_R)
        # self.layer_norm2 = LayerNorm(config.Feature1_SIZE)

        self.dropout3 = nn.Dropout(config.DROPOUT_R)
        self.layer_norm3 = LayerNorm(config.Feature1_SIZE)

    def forward(self, x, y, x_mask, y_mask):
        x = self.layer_norm1(x + self.dropout1(
            self.multi_head_attention1(x, y, x, x_mask)
        ))
        # x = self.layer_norm2(x + self.dropout2(
        #     self.multi_head_attention2(x, y, x, y_mask)
        # ))
        x = self.layer_norm3(x + self.dropout3(
            self.feed_forward(x)
        ))

        return x

In [5]:
# Configure parameters
class Config:
    def __init__(self):
        self.HIDDEN_SIZE = 512
        self.FF_SIZE = 512
        self.DROPOUT_R = 0.1
        self.MULTI_HEAD = 8
        assert self.HIDDEN_SIZE % self.MULTI_HEAD == 0
        self.HIDDEN_SIZE_HEAD = int(self.HIDDEN_SIZE / self.MULTI_HEAD)
        self.Feature1_SIZE = spatial[0].shape[1]
        self.Feature2_SIZE = spatial[0].shape[1]

# Initialize configuration
config = Config()

# Initialize SGA module
sga_module = SGA(config)

# Generate masks
def make_mask(feature):
    return (torch.sum(torch.abs(feature), dim=-1) == 0).unsqueeze(1).unsqueeze(2)


In [6]:
import gc
with torch.no_grad():
    gc.collect()
    torch.cuda.empty_cache()
    X1 = spatial[0].unsqueeze(0).float().to('cuda')
    X2 = spatial[1].unsqueeze(0).float().to('cuda')
    x_mask = make_mask(X1.float()).to('cuda')
    y_mask = make_mask(X2.float()).to('cuda')
    sga_module.to('cuda')
    current_input = sga_module(X1, X2, x_mask, y_mask)
    print(current_input)
    for i in range(2,len(spatial)):
        # del X1,X2,x_mask,y_mask
        gc.collect()
        torch.cuda.empty_cache()
        X1 = current_input
        X2 = spatial[i].unsqueeze(0).float().to('cuda')
        x_mask = make_mask(X1.float()).to('cuda')
        y_mask = make_mask(X2.float()).to('cuda')
        current_input = sga_module(X1, X2, x_mask, y_mask)
        print(current_input)
current_input

tensor([[[-0.8538, -0.6988,  0.7390,  ...,  1.3109, -0.4787,  1.0033],
         [-0.9348, -0.5899,  0.7842,  ...,  1.4919, -0.3146,  1.1863],
         [-0.0964, -0.7644, -0.2824,  ...,  1.4550, -0.4800,  0.9489],
         ...,
         [-0.8703, -0.8020,  0.5409,  ...,  1.3173, -0.0541,  1.0655],
         [-1.1832, -0.9378,  0.5809,  ...,  1.5389, -0.4571,  1.1874],
         [-0.9058, -0.7987,  0.5589,  ...,  1.5868, -0.3842,  1.1088]]],
       device='cuda:0')


tensor([[[-0.8538, -0.6988,  0.7390,  ...,  1.3109, -0.4787,  1.0033],
         [-0.9348, -0.5899,  0.7842,  ...,  1.4919, -0.3146,  1.1863],
         [-0.0964, -0.7644, -0.2824,  ...,  1.4550, -0.4800,  0.9489],
         ...,
         [-0.8703, -0.8020,  0.5409,  ...,  1.3173, -0.0541,  1.0655],
         [-1.1832, -0.9378,  0.5809,  ...,  1.5389, -0.4571,  1.1874],
         [-0.9058, -0.7987,  0.5589,  ...,  1.5868, -0.3842,  1.1088]]],
       device='cuda:0')

In [7]:
current_input.shape

torch.Size([1, 5592, 100])

In [8]:
import pandas as pd
reshaped_tensor = torch.squeeze(current_input).view(5592, 100)
numpy_array = reshaped_tensor.cpu().numpy()
final = pd.DataFrame(numpy_array)
final

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.853835,-0.698774,0.738998,-0.045378,-0.116367,0.174600,-0.243816,2.487399,0.558009,-0.727456,...,-0.016880,0.731542,0.991435,0.381862,0.668487,0.585468,0.179599,1.310859,-0.478688,1.003326
1,-0.934829,-0.589923,0.784242,0.264519,-0.170699,0.268877,-0.120890,0.523225,0.630536,-0.516873,...,0.015461,0.813827,0.965184,0.651837,0.838446,0.639399,0.256180,1.491901,-0.314560,1.186334
2,-0.096383,-0.764445,-0.282430,-0.037582,-0.120513,0.310358,-0.198896,2.437239,0.627342,-0.698358,...,-0.346342,0.771247,1.014533,0.381136,0.739023,0.543372,0.222055,1.454999,-0.480026,0.948938
3,-0.866893,-0.325123,0.549122,0.118293,-0.192852,-0.129002,0.000151,2.382843,0.510228,-0.468551,...,0.042357,0.696352,0.991708,0.445259,-0.506932,0.471720,0.400520,1.545528,-0.460037,1.163922
4,-0.937585,-0.714162,0.638218,0.559154,-0.285168,0.150995,-0.559380,0.436686,0.510724,-0.669533,...,0.285589,0.623705,0.910474,0.604349,0.612439,0.528975,0.317026,1.504469,-0.524921,1.272971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5587,0.079247,0.216752,-1.184817,-0.284532,-1.173024,-0.463248,-1.385232,1.532151,2.073771,-0.774980,...,-0.571327,1.279978,-0.785562,-0.741090,-1.308580,-0.615403,2.086149,1.802725,-0.201333,-0.485090
5588,-0.980605,-0.892185,0.553306,0.458857,-0.152461,0.172069,-0.105800,2.463047,0.318549,-0.713937,...,0.125182,0.817362,0.993720,0.550011,0.759665,0.535267,0.370594,1.611545,-0.517124,1.205186
5589,-0.870327,-0.801968,0.540880,-0.024898,-0.126186,0.112049,-0.544033,2.415838,0.518048,-0.705705,...,-0.025727,0.652586,1.009885,0.526750,1.092181,0.715187,0.118085,1.317252,-0.054133,1.065456
5590,-1.183180,-0.937780,0.580935,-0.728721,-0.286498,0.166919,-0.546322,0.591584,0.009069,-0.489089,...,-0.240434,0.732498,1.003324,0.818964,0.811158,0.667322,0.160316,1.538945,-0.457087,1.187384


In [9]:
final.to_csv("final_spatial_fusion_GEO.csv",index=None,columns=None)